# Sequential Layer

This notebook walks through how to build a sequential layer type, allowing you to chain an arbitrary number of layers of the same type together.

In [1]:
%install '.package(path: "$cwd/FastaiNotebook_00_load_data")' FastaiNotebook_00_load_data

Installing packages:
	.package(path: "/home/saeta/fastai_docs/dev_swift/FastaiNotebook_00_load_data")
		FastaiNotebook_00_load_data
With SwiftPM flags: []
Working in: /tmp/tmpc0cexds2
Fetching https://github.com/mxcl/Path.swift
Fetching https://github.com/JustHTTP/Just
Completed resolution in 1.29s
Cloning https://github.com/JustHTTP/Just
Resolving https://github.com/JustHTTP/Just at 0.7.1
Cloning https://github.com/mxcl/Path.swift
Resolving https://github.com/mxcl/Path.swift at 0.16.2
Compile Swift Module 'Path' (9 sources)
Compile Swift Module 'Just' (1 sources)
Compile Swift Module 'FastaiNotebook_00_load_data' (1 sources)
Compile Swift Module 'jupyterInstalledPackages' (1 sources)
Linking ./.build/x86_64-unknown-linux/debug/libjupyterInstalledPackages.so
Initializing Swift...
Loading library...
Installation complete!


In [2]:
import FastaiNotebook_00_load_data

In [3]:
// export

/// Define a new Differentiable data type that will be the AllDifferentiableVariables, Cotangent-, and Tangent vectors
/// for our seuqential layer type.
public struct DiffList<U: Differentiable & AdditiveArithmetic & Equatable & VectorNumeric>: KeyPathIterable {
  public var u: [U] = []
}

In [4]:
// export
extension DiffList: Equatable {
  public static func == (lhs: DiffList, rhs: DiffList) -> Bool {
    if lhs.u.count != rhs.u.count {
      return false
    }

    for i in 0..<lhs.u.count {
      if lhs.u[i] != rhs.u[i] {
        return false
      }
    }

    return true
  }
}


In [5]:
// export

// Note: it'd be good to split up this cell into multiple, but there's a circular dependency between adding
// derivatives to the AdditiveArithmetic protocol, and marking the type as differentiable. Retroactive conformances
// would probably help here.

extension DiffList: AdditiveArithmetic {
  public static var zero: DiffList {
    get {
      return DiffList()
    }
  }

  @differentiable(vjp: _vjpAdd(lhs:rhs:))
  public static func + (lhs: DiffList, rhs: DiffList) -> DiffList {
    precondition(lhs.u.count == 0 || rhs.u.count == 0 || lhs.u.count == rhs.u.count,
        "DiffList size mis-match: lhs: \(lhs.u.count), rhs: \(rhs.u.count)")
    if lhs.u.count == 0 {
      return rhs
    }
    if rhs.u.count == 0 {
      return lhs
    }
    var output: [U] = []
    for i in 0..<lhs.u.count {
      output.append(lhs.u[i] + rhs.u[i])
    }
    return DiffList(u: output)
  }

  public static func _vjpAdd(lhs: DiffList, rhs: DiffList) -> (DiffList, (DiffList) -> (DiffList, DiffList)) {
    return (lhs + rhs, { [lhsCount = lhs.u.count, rhsCount = rhs.u.count] v in
        precondition(v.u.count == lhsCount || lhsCount == 0,
            "DiffList gradient size mis-match: v: \(v.u.count), lhs: \(lhsCount)")
        precondition(v.u.count == rhsCount || rhsCount == 0,
            "DiffList gradient size mis-match: v: \(v.u.count), rhs: \(rhsCount)")

        var lhsOutput: [U]  = []
        var rhsOutput: [U]  = []

        // Unbroadcasting
        if lhsCount != 0 {
          lhsOutput = v.u
        }
        if rhsCount != 0 {
          rhsOutput = v.u
        }

        return (DiffList(u: lhsOutput), DiffList(u: rhsOutput))
    })
  }

  @differentiable(vjp: _vjpSubtract(lhs:rhs:))
  public static func - (lhs: DiffList, rhs: DiffList) -> DiffList {
    precondition(lhs.u.count == 0 || rhs.u.count == 0 || lhs.u.count == rhs.u.count,
        "DiffList size mis-match: lhs: \(lhs.u.count), rhs: \(rhs.u.count)")
    if lhs.u.count == 0 {
      return rhs
    }
    if rhs.u.count == 0 {
      return lhs
    }
    var output: [U] = []
    for i in 0..<lhs.u.count {
      output.append(lhs.u[i] + rhs.u[i])
    }
    return DiffList(u: output)
  }

  public static func _vjpSubtract(lhs: DiffList, rhs: DiffList) -> (DiffList, (DiffList) -> (DiffList, DiffList)) {
    return (lhs + rhs, { [lhsCount = lhs.u.count, rhsCount = rhs.u.count] v in
        precondition(v.u.count == lhsCount || lhsCount == 0,
            "DiffList gradient size mis-match: v: \(v.u.count), lhs: \(lhsCount)")
        precondition(v.u.count == rhsCount || rhsCount == 0,
            "DiffList gradient size mis-match: v: \(v.u.count), rhs: \(rhsCount)")

        var lhsOutput: [U]  = []
        var rhsOutput: [U]  = []

        // Unbroadcasting
        if lhsCount != 0 {
          lhsOutput = v.u
        }
        if rhsCount != 0 {
          rhsOutput = v.u.map({ U.zero - $0 })
        }

        return (DiffList(u: lhsOutput), DiffList(u: rhsOutput))
    })
  }
}

extension DiffList: VectorNumeric {
  public typealias Scalar = U.Scalar

  public static func * (lhs: Scalar, rhs: DiffList) -> DiffList {
    return DiffList(u: rhs.u.map( { $0 * lhs } ))
  }
}

extension DiffList: Differentiable {
  public typealias TangentVector = DiffList
  public typealias CotangentVector = DiffList
  public typealias AllDifferentiableVariables = DiffList

  public func tangentVector(from cotangent: CotangentVector) -> TangentVector {
    return cotangent
  }

}


In [6]:
// export
import TensorFlow  // Defines Layer.

In [7]:
// export

/// A struct that contains a number of layers within it.
public struct SequentialLayer<U: Layer>: KeyPathIterable where 
    U.Input == U.Output,
    U.AllDifferentiableVariables: VectorNumeric,
    U.AllDifferentiableVariables == U.CotangentVector {

    public var layers: [U]

    public init(layers: [U]) {
        self.layers = layers
    }
}


In [8]:
// export

// Not strictly necessary, but nice to have.

extension SequentialLayer: Equatable where U: Equatable {
  public static func ==(lhs: SequentialLayer, rhs: SequentialLayer) -> Bool {
    return lhs.layers == rhs.layers
  }
}


In [9]:
// export

// Mark SequentialLayer as Differentiable

extension SequentialLayer: Differentiable {
  public typealias AllDifferentiableVariables = DiffList<U.AllDifferentiableVariables>
  public typealias TangentVector = DiffList<U.TangentVector>
  public typealias CotangentVector = DiffList<U.CotangentVector>

  public func tangentVector(from cotangent: CotangentVector) -> TangentVector {
    precondition(cotangent.u.count == layers.count, "Differing # of layers: \(cotangent.u.count) and \(layers.count)")
    return DiffList(u: zip(layers, cotangent.u).map({ $0.0.tangentVector(from: $0.1) }))
  }

  public func moved(along direction: TangentVector) -> SequentialLayer {
    precondition(direction.u.count == layers.count, "Differing # of layers: \(direction.u.count) and \(layers.count)")
    return SequentialLayer(layers: zip(layers, direction.u).map({ $0.0.moved(along: $0.1) }))
  }

  public var allDifferentiableVariables: AllDifferentiableVariables {
    get {
      return DiffList(u: layers.map({ $0.allDifferentiableVariables }))
    }
    set {
      precondition(newValue.u.count == layers.count, "Differing # of layers: \(newValue.u.count) and \(layers.count)")
      for i in 0..<layers.count {
        layers[i].allDifferentiableVariables = newValue.u[i]
      }
    }
  }
}


In [10]:
// export

// Conform SequentialLayer to the Layer protocol

extension SequentialLayer: Layer {

    public typealias Input = U.Input
    public typealias Output = U.Output
  
    @differentiable(vjp: _appliedDifferentiating(to:in:))
    public func applied(to input: Input, in context: Context) -> Output {
        var tmp = input
        for layer in layers {
          tmp = layer.applied(to: tmp, in: context)
        }
        return tmp
    }
   
    public func _appliedDifferentiating(to input: Input, in context: Context) -> (
        Output, (Output.CotangentVector) -> (CotangentVector, Input.CotangentVector)) {
        
        var pullbacks: [(U.Output.CotangentVector) -> (U.AllDifferentiableVariables, U.Input.CotangentVector)] = []
        var tmp = input
        for layer in layers {
          let (output, pullback) = Swift.valueWithPullback(at: layer, tmp) { layer, input in
            return layer.applied(to: input, in: context)
          }
          tmp = output
          pullbacks.append(pullback)
        }
        
        return (tmp, { input in
                    var allDiffVars: [U.AllDifferentiableVariables] = []
                    var tmp = input
                    
                    for pb in pullbacks.reversed() {
                        let (diffVars, input) = pb(tmp)
                        tmp = input
                        allDiffVars.append(diffVars)
                    }
                      
                    return (DiffList(u: allDiffVars.reversed()), tmp)
                })
    }
}


In [11]:
struct MyModel: Layer {
    var layers: SequentialLayer<Dense<Float>>
    
    init(inputSize: Int, hiddenUnits: [Int], outputSize: Int) {
        // Make the dense layers.
        
        // TODO(saeta): Clean up this code.
        var input = inputSize
        var output = outputSize
        if hiddenUnits.count > 0 {
            output = hiddenUnits[0]
        }
        var layers: [Dense<Float>] = []
        for i in 0..<hiddenUnits.count {
            output = hiddenUnits[i]
            print("Making Dense<Float>(inputSize: \(input), outputSize: \(output))")
            layers.append(Dense<Float>(inputSize: input, outputSize: output))
            input = output
        }
        print("Making Dense<Float>(inputSize: \(output), outputSize: \(outputSize))")
        layers.append(Dense<Float>(inputSize: output, outputSize: outputSize))
        
        self.layers = SequentialLayer(layers: layers)
    }
    
    @differentiable
    func applied(to input: Tensor<Float>, in context: Context) -> Tensor<Float> {
        return layers.applied(to: input, in: context)
    }
}

In [12]:
struct FixedModel: Layer {
    var d1 = Dense<Float>(inputSize: 784, outputSize: 30)
    var d2 = Dense<Float>(inputSize: 30, outputSize: 10)
    
    @differentiable
    func applied(to input: Tensor<Float>, in context: Context) -> Tensor<Float> {
        return input.sequenced(in: context, through: d1, d2)
    }
}

In [13]:
var (xTrain, yTrain, xValid, yValid) = loadMNIST(path: mnistPath, flat: true)

In [14]:
func logSumExp<Scalar>(_ x: Tensor<Scalar>) -> Tensor<Scalar> where Scalar:TensorFlowFloatingPoint{
    let m = x.max(alongAxes: -1)
    return m + log(exp(x-m).sum(alongAxes: -1))
}
func logSoftmax<Scalar>(_ activations: Tensor<Scalar>) -> Tensor<Scalar> where Scalar:TensorFlowFloatingPoint{
    return activations - logSumExp(activations)
}

In [15]:
let trainingContext = Context(learningPhase: .training)

In [16]:
let lr:Float = 0.5   // learning rate
let epochs = 1      // how many epochs to train for
let bs:Int32=64                         // batch size
let (n,m) = (60000,784)  // MNIST dataset size


In [17]:
var modelFixed = FixedModel()
let modelFixedStart = modelFixed

In [18]:
modelFixedStart.d1.weight == modelFixed.d1.weight

true


In [19]:
var modelFlex = MyModel(inputSize: 784, hiddenUnits: [30], outputSize: 10)
let modelFlexStart = modelFlex

Making Dense<Float>(inputSize: 784, outputSize: 30)
Making Dense<Float>(inputSize: 30, outputSize: 10)


In [20]:
modelFlex.layers.layers[0].weight == modelFlexStart.layers.layers[0].weight

true


In [21]:
public func accuracy(_ output: Tensor<Float>, _ target: Tensor<Int32>) -> Tensor<Float>{
    let corrects = Tensor<Float>(output.argmax(squeezingAxis: 1) .== target)
    return corrects.mean()
}

In [22]:
let inferenceContext = Context(learningPhase: .inference)

In [23]:
accuracy(modelFlex.applied(to: xValid, in: inferenceContext), yValid)

0.1409


In [24]:
accuracy(modelFixed.applied(to: xValid, in: inferenceContext), yValid)

0.1388


The training loops below are copied from 03_minibatch. They don't appear to actually train either model. :-(

In [ ]:
for epoch in 1...epochs{
    for i in 0..<((n-1)/Int(bs)){
        let startIdx = Int32(i) * bs
        let endIdx = startIdx + bs
        let xb = xTrain[startIdx..<endIdx]
        let yb = yTrain[startIdx..<endIdx]
        let (loss, grads) = modelFixed.valueWithGradient { model -> Tensor<Float> in
            let preds = model.applied(to: xb, in: trainingContext)
            return softmaxCrossEntropy(logits: preds, labels: yb)
        }
        var parameters = modelFixed.allDifferentiableVariables
        for kp in parameters.recursivelyAllWritableKeyPaths(to: Tensor<Float>.self){ 
            parameters[keyPath: kp] -= lr * grads[keyPath:kp]
        }
    }
}

In [ ]:
for epoch in 1...epochs{
    for i in 0..<((n-1)/Int(bs)){
        let startIdx = Int32(i) * bs
        let endIdx = startIdx + bs
        let xb = xTrain[startIdx..<endIdx]
        let yb = yTrain[startIdx..<endIdx]
        let (loss, grads) = modelFlex.valueWithGradient { model -> Tensor<Float> in
            let preds = model.applied(to: xb, in: trainingContext)
            return softmaxCrossEntropy(logits: preds, labels: yb)
        }
        var parameters = modelFlex.allDifferentiableVariables
        for kp in parameters.recursivelyAllWritableKeyPaths(to: Tensor<Float>.self){ 
            parameters[keyPath: kp] -= lr * grads[keyPath: kp]
        }
    }
}

In [25]:
accuracy(modelFlex.applied(to: xValid, in: inferenceContext), yValid)

0.1409


In [26]:
accuracy(modelFixed.applied(to: xValid, in: inferenceContext), yValid)

0.1388


In [27]:
modelFixedStart.d1.weight == modelFixed.d1.weight

true


In [28]:
modelFlex.layers.layers[0].weight == modelFlexStart.layers.layers[0].weight

true


In [29]:
let optimizerFixed = SGD<FixedModel, Float>(learningRate: lr)
// let optimizerFlex = SGD<MyModel, Float>(learningRate: lr)  // SGD doesn't work for the new flex style models, due to the interaction between how .zero is defined, and keypathing.

In [30]:
/// Stochastic gradient descent (SGD) optimizer.
///
/// An optimizer that implements stochastic gradient descent, with support for momentum, learning
/// rate decay, and Nesterov momentum.
public class SimpleSGD<Model: Layer, Scalar: TensorFlowFloatingPoint>: Optimizer
    where Model.AllDifferentiableVariables == Model.CotangentVector {
    /// The learning rate.
    public var learningRate: Scalar

    public init(
        learningRate: Scalar = 0.01
    ) {
        precondition(learningRate >= 0, "Learning rate must be non-negative")

        self.learningRate = learningRate
    }

    public func update(_ model: inout Model.AllDifferentiableVariables,
                       along direction: Model.CotangentVector) {
        for kp in model.recursivelyAllWritableKeyPaths(to: Tensor<Scalar>.self) {
            model[keyPath: kp] -= -learningRate * direction[keyPath: kp]
        }
    }
}


In [31]:
let simpleOptFlex = SimpleSGD<MyModel, Float>(learningRate: lr)

In [32]:
for epoch in 1...epochs{
    for i in 0..<((n-1)/Int(bs)){
        let startIdx = Int32(i) * bs
        let endIdx = startIdx + bs
        let xb = xTrain[startIdx..<endIdx]
        let yb = yTrain[startIdx..<endIdx]
        let (loss, grads) = modelFixed.valueWithGradient { model -> Tensor<Float> in
            let preds = model.applied(to: xb, in: trainingContext)
            return softmaxCrossEntropy(logits: preds, labels: yb)
        }
        optimizerFixed.update(&modelFixed.allDifferentiableVariables, along: grads)
    }
}

In [33]:
accuracy(modelFixed.applied(to: xValid, in: inferenceContext), yValid)

0.098


In [34]:
modelFixedStart.d1.weight == modelFixed.d1.weight

false


In [35]:
for epoch in 1...epochs{
    for i in 0..<((n-1)/Int(bs)){
        let startIdx = Int32(i) * bs
        let endIdx = startIdx + bs
        let xb = xTrain[startIdx..<endIdx]
        let yb = yTrain[startIdx..<endIdx]
        let (loss, grads) = modelFlex.valueWithGradient { model -> Tensor<Float> in
            let preds = model.applied(to: xb, in: trainingContext)
            return softmaxCrossEntropy(logits: preds, labels: yb)
        }
//        optimizerFlex.update(&modelFlex.allDifferentiableVariables, along: grads)
        simpleOptFlex.update(&modelFlex.allDifferentiableVariables, along: grads)
    }
}

In [36]:
accuracy(modelFlex.applied(to: xValid, in: inferenceContext), yValid)

0.098


In [37]:
modelFlex.layers.layers[0].weight == modelFlexStart.layers.layers[0].weight

false
